<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#uberduck_ml_dev.exec.select_speakers" data-toc-modified-id="uberduck_ml_dev.exec.select_speakers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>uberduck_ml_dev.exec.select_speakers</a></span></li></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#uberduck_ml_dev.exec.select_speakers" data-toc-modified-id="uberduck_ml_dev.exec.select_speakers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>uberduck_ml_dev.exec.select_speakers</a></span></li></ul></div>

In [ ]:
# default_exp exec.select_speakers

# uberduck_ml_dev.exec.select_speakers

In [ ]:
# export
import argparse
from collections import namedtuple
from dataclasses import dataclass
import json
import os
from pathlib import Path
from shutil import copyfile, copytree
import sys
from typing import List, Optional, Set

import sqlite3
from tqdm import tqdm

from uberduck_ml_dev.data.cache import ensure_filelist_in_cache, ensure_speaker_table
from uberduck_ml_dev.utils.audio import convert_to_wav
from uberduck_ml_dev.utils.utils import parse_vctk

CACHE_LOCATION = Path.home() / Path(".cache/uberduck/uberduck-ml-dev.db")


@dataclass
class Filelist:
    path: str
    sql: Optional[str] = None
    speaker_ids: Optional[List[int]] = None
    speakers: Optional[List[str]] = None
    speaker_idx_in_path: int = None


def _get_speaker_ids(filelist: Filelist) -> Set[int]:
    if filelist.speaker_ids:
        return set(filelist.speaker_ids)

    path = os.path.expanduser(filelist.path)
    ensure_speaker_table()
    ensure_filelist_in_cache(path, speaker_idx_in_path=filelist.speaker_idx_in_path)
    if filelist.speakers:
        # conn =
        conn = sqlite3.connect(str(CACHE_LOCATION))
        cursor = conn.cursor()
        params = ",".join("?" for _ in filelist.speakers)
        results = cursor.execute(
            f"SELECT speaker_id FROM speakers where filepath = ? AND name in ({params})",
            [path, *filelist.speakers],
        ).fetchall()
        speaker_ids = set([speaker_id for (speaker_id, *_) in results])
        return speaker_ids
    elif filelist.sql:
        if not CACHE_LOCATION.exists():
            msg = "Filelist cache does not exist! You must generate it."
            print(msg)
            raise Exception(msg)
        conn = sqlite3.connect(str(CACHE_LOCATION))
        cursor = conn.cursor()
        results = cursor.execute(filelist.sql).fetchall()

        speaker_ids = set([speaker_id for (speaker_id, *_) in results])
        return speaker_ids


def select_speakers(filelists: List[Filelist], output_filelist: str):
    new_speaker_id = 0
    seen_speaker_ids = dict()
    with open(output_filelist, "w") as f_out:
        for filelist in tqdm(filelists):
            speaker_ids = _get_speaker_ids(filelist)
            if filelist.path:
                filelist.path = os.path.expanduser(filelist.path)
            with open(filelist.path, "r") as f_in:
                for line in f_in.readlines():
                    path, txn, original_speaker_id = line.strip().split("|")
                    if (
                        speaker_ids is not None
                        and int(original_speaker_id) not in speaker_ids
                    ):
                        continue
                    if (filelist.path, original_speaker_id) not in seen_speaker_ids:
                        seen_speaker_ids[
                            (filelist.path, original_speaker_id)
                        ] = new_speaker_id
                        current_speaker_id = new_speaker_id
                        new_speaker_id += 1
                    else:
                        current_speaker_id = seen_speaker_ids[
                            (filelist.path, original_speaker_id)
                        ]
                    f_out.write(f"{path}|{txn}|{current_speaker_id}\n")


def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", help="path to JSON config")
    return parser.parse_args(args)


try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False

if __name__ == "__main__" and not IN_NOTEBOOK:
    args = parse_args(sys.argv[1:])
    if args.config:
        with open(args.config) as f:
            config = json.load(f)
        filelists = [Filelist(**f) for f in config["filelists"]]
        output_filelist = config["output"]
    else:
        raise Exception("You must pass a config file!")
    select_speakers(filelists, output_filelist)

In [ ]:
# skip
_get_speaker_ids(
    Filelist(
        "/Users/zwf/data/voice/dvc-managed/uberduck-multispeaker/list.txt",
        speakers=["Carl_Wheezer"],
        speaker_idx_in_path=-1,
    ),
)

Filelist not found in cache. Cache count: 5404. Filelist count: 73.


{41}

In [ ]:
args = parse_args(["--config", "foo.json"])
assert args.config == "foo.json"

In [ ]:
# sam stuff
import sqlite3
import os
import pandas as pd


def get_filelist(database, speakerjson):
    """
    Take a list of speakers and create a filelist
    """


def subset_speakers(database, seed, speakerlist=None, nspeakers=None):
    """
    Takes a filelist and saves another filelist with either a random subset of speakers or speakers from the list
    """